### Code Description

This code prepares the model's training datasets. The output format is JSON Lines (jsonl), ideal for training models with large datasets, while keeping the data structure lightweight and easy to process.

### Imports

In [5]:
import pandas as pd
import json
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import os

### Dataset Global

In [ ]:
all_data = pd.read_csv('../database/combined_data.csv', sep=";")

state_product_dict = {
    state: list(all_data[all_data['state'] == state]['product'].unique())
    for state in all_data['state'].unique()
}

output_file = 'dataset_global/dataset_global.jsonl'

os.makedirs(os.path.dirname(output_file), exist_ok=True)

# Open the output file in write mode
with open(output_file, 'w') as file:
    
    # ''' 
    # # INFO: ======== Raw Data ========
    # ''' 
    # for state, products in state_product_dict.items():
    #     for product in products:

    #         # Filter data for the current state and product
    #         data_filtered = all_data[(all_data['state'] == state) & (all_data['product'] == product)]

    #         sequence = data_filtered['m3'].tolist()
    #         json_line = {f'{product}_{state}': sequence}

    #         file.write(json.dumps(json_line) + '\n')

    ''' 
    # INFO: ======== Remove SP ========
    ''' 
    for state, products in state_product_dict.items():
        if state == "sp":
            continue

        for product in products:
            
            # Filter data for the current state and product
            data_filtered = all_data[(all_data['state'] == state) & (all_data['product'] == product)]

            sequence = data_filtered['m3'].tolist()
            json_line = {f'{product}_{state}': sequence}

            file.write(json.dumps(json_line) + '\n')
    
    ''' 
    # INFO: ======== MinMaxScaler ========
    ''' 
    # for state, products in state_product_dict.items():
    #     for product in products:
    #         data_filtered = all_data[(all_data['state'] == state) & (all_data['product'] == product)]
            
    #         data = rolling_window(data_filtered['m3'], 12)
    #         print(data)

    #         sequence = data.values  

    #         print(sequence)
            
    #         scaler = MinMaxScaler(feature_range=(-1, 1))
    #         sequence_scaled = scaler.fit_transform(sequence.reshape(-1, 1)).flatten()
    #         print(sequence_scaled)
            
    #         json_line = {"sequence": sequence_scaled.tolist()} 
            
    #         file.write(json.dumps(json_line) + '\n')
    

print(f"Filtered data has been saved to {output_file}")

pe
pe
pe
pe
pe
pe
pe
pe
ba
ba
ba
ba
ba
ba
ba
ba
ro
ro
ro
ro
ro
ro
ro
ro
es
es
es
es
es
es
es
es
pi
pi
pi
pi
pi
pi
pi
pi
to
to
to
to
to
to
to
to
ma
ma
ma
ma
ma
ma
ma
ma
pr
pr
pr
pr
pr
pr
pr
pr
ap
ap
ap
ap
ap
ap
ap
ap
sc
sc
sc
sc
sc
sc
sc
sc
mt
mt
mt
mt
mt
mt
mt
mt
rs
rs
rs
rs
rs
rs
rs
rs
rr
rr
rr
rr
rr
rr
rr
rr
rn
rn
rn
rn
rn
rn
rn
rn
rj
rj
rj
rj
rj
rj
rj
rj
df
df
df
df
df
df
df
df
pb
pb
pb
pb
pb
pb
pb
pb
ms
ms
ms
ms
ms
ms
ms
ms
mg
mg
mg
mg
mg
mg
mg
mg
al
al
al
al
al
al
al
al
se
se
se
se
se
se
se
se
ac
ac
ac
ac
ac
ac
ac
ac
pa
pa
pa
pa
pa
pa
pa
pa
am
am
am
am
am
am
am
am
go
go
go
go
go
go
go
go
ce
ce
ce
ce
ce
ce
ce
ce
Filtered data has been saved to dataset_global_sp/dataset_global_sp.jsonl
